In [2]:
import torch
import cv2
import sys
import numpy as np
import argparse
import torch.nn as nn
from google.colab import drive
from torch.autograd import Variable
from torch.autograd import Function
from torchvision import models
from torchvision import utils

In [3]:
!pip install memory_profiler

  Created wheel for memory-profiler: filename=memory_profiler-0.58.0-cp37-none-any.whl size=30180 sha256=c96d968ff41d792c94f3c3b0b165e317c8b557f905b6e79d72c6b60135006ceb
  Stored in directory: /root/.cache/pip/wheels/02/e4/0b/aaab481fc5dd2a4ea59e78bc7231bb6aae7635ca7ee79f8ae5
Successfully built memory-profiler


In [4]:
%load_ext memory_profiler

In [5]:
# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...


## Import depedences from google drive

In [6]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [7]:
%cd '/content/gdrive/My Drive/UnB/TCC-1/TCC1-1-dataset-final'

/content/gdrive/My Drive/UnB/TCC-1/TCC1-1-dataset-final


In [8]:
!ls

cm_data_set_exp1.png  intrepretabilidade_results	skin.txt
dataset_final	      main.py				train_val_phase.csv
dataset_final.tar.xz  __pycache__
grad_cam.py	      restnet_model152_trained_exp7.pt


## Load the skin lession model

In [9]:
PRE_MODEL_DIR='/content/gdrive/My Drive/UnB/TCC-1/TCC1-1-dataset-final/restnet_model152_trained_exp7.pt'

In [10]:
model_name='resnet'
num_classes = 9
feature_extract = False

In [11]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [12]:
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0

    if model_name == "resnet":
        """ Resnet152
        """
        model_ft = models.resnet152(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    else:
        print("Invalid model name, exiting...")
        exit()
    
    return model_ft, input_size

# Initialize the model for this run
model, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)

Downloading: "https://download.pytorch.org/models/resnet152-b121ed2d.pth" to /root/.cache/torch/hub/checkpoints/resnet152-b121ed2d.pth


In [13]:
if train_on_gpu:
    state = torch.load(PRE_MODEL_DIR)
else:
    state = torch.load(PRE_MODEL_DIR, map_location='cpu')

# Loading weights in restnet architecture
model.load_state_dict(state['state_dict'])

<All keys matched successfully>

In [14]:
classes_skin = state['class_to_idx']
classes_skin

{'actinic-keratosis': 0,
 'basal-cell-carcinoma': 1,
 'dermatofibroma': 2,
 'hemangioma': 3,
 'intraepithelial-carcinoma': 4,
 'malignant-melanoma': 5,
 'melanocytic-nevus': 6,
 'pyogenic-granuloma': 7,
 'squamous-cell-carcinoma': 8}

In [15]:
skin_list = list(classes_skin.keys())
skin_list

['actinic-keratosis',
 'basal-cell-carcinoma',
 'dermatofibroma',
 'hemangioma',
 'intraepithelial-carcinoma',
 'malignant-melanoma',
 'melanocytic-nevus',
 'pyogenic-granuloma',
 'squamous-cell-carcinoma']

In [16]:
from torchvision import datasets, transforms, models

test_dir = '/content/gdrive/My Drive/UnB/TCC-1/TCC1-1-dataset-final/dataset_final/test'

mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

test_transforms = transforms.Compose([transforms.Resize(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize(mean,
                                      std)
                                      ])

# TODO: Load the datasets with ImageFolder
test_data = datasets.ImageFolder(test_dir,transform = test_transforms)

# TODO: Using the image datasets and the trainforms, define the dataloaders
testloader = torch.utils.data.DataLoader(test_data, batch_size=1, shuffle=True)

In [17]:
def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    
    # Resize where shortest side is 256px, keeping aspect ratio
    minside = 256
    img = Image.open(image)
    imagex, imagey = img.size
    aspect = float(imagex)/float(imagey)
    
    if imagex <= imagey:
      width = 256
      height = int(width/aspect)
    else:
      height = 256
      width = int(height*aspect)
   
    img = img.resize((width,height),Image.ANTIALIAS)
    
    # Crop out center 224 x 224
    left = (width - 224)/2
    top = (height - 224)/2
    right = (width + 224)/2
    bottom = (height + 224)/2
    img = img.crop((left, top, right, bottom))
    
    # Convert image to numpy array
    np_image = np.array(img)
    np_image = np_image/255
    
    # Normalize image
    np_image -= [0.485, 0.456, 0.406]
    np_image /= [0.229, 0.224, 0.225]
    
    # Transpose array:
    result = np_image.transpose(-1,0,1)
    
    return result


In [18]:
def plot_bar(image_path, model, save_output):
  result = process_image(image_path)
  res = torch.from_numpy(result)
  fig, (ax1, ax2) = plt.subplots(figsize=(6,9), ncols=2)
  ax1 = imshow(res, ax1)
  probs, classes = predict(image_path,model)
  ax2.barh(np.arange(len(probs)), probs)
  ax2.set_aspect(0.1)
  ax2.set_yticks(np.arange(len(probs)))
  v = list(skin_list)
  # classes = list(map(lambda x: skin_lesion_to_name[x], classes))
  ax2.set_yticklabels(classes, size='small');
  ax2.set_title('Class Probability')
  ax2.set_xlim(0, 1.1)
  plt.tight_layout()
  plt.savefig(save_output)

In [19]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.numpy().transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

In [20]:
def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    
    # TODO: Implement the code to predict the class from an image file
    model.eval()
    image = process_image(image_path)
    image = torch.from_numpy(image)

    if train_on_gpu:
      model.cuda()
      image = image.cuda()
    image = image.float().unsqueeze(0)
    out = model.forward(image)
    logps = F.log_softmax(out)
    ps = torch.exp(logps)
    probs, classes = ps.topk(topk, dim=1)

    if train_on_gpu:
      probs = list(probs.squeeze(0).cpu().detach().numpy())
      classes = list(classes.squeeze(0).cpu().detach().numpy())

    else:
      probs = list(probs.squeeze(0).detach().numpy())
      classes = list(classes.squeeze(0).detach().numpy())
    idx_class_mapping = dict((v,k) for k,v in test_data.class_to_idx.items())
    classes = list(map(lambda x: idx_class_mapping[x], classes))

    return probs, classes


In [21]:
from PIL import Image
import matplotlib.pyplot as plt
import torch.nn.functional as F


probs, classes = predict('/content/gdrive/My Drive/UnB/TCC-1/TCC1-1-dataset-final/dataset_final/test/melanocytic-nevus/worst.jpg', model)
print(probs)
print(classes)

[0.9416537, 0.058291752, 4.8728944e-05, 2.88077e-06, 2.0546156e-06]
['hemangioma', 'pyogenic-granuloma', 'basal-cell-carcinoma', 'melanocytic-nevus', 'squamous-cell-carcinoma']


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  from ipykernel import kernelapp as app


In [ ]:
#plot_bar('/content/gdrive/My Drive/UnB/TCC-1/TCC1-1-dataset-final/dataset_final/test/melanocytic-nevus/worst.jpg', model, '/content/gdrive/My Drive/UnB/TCC-1/TCC1-1-dataset-final/intrepretabilidade_results/TCC2/teste')

## Main function

In [ ]:
#!python main.py demo1 -a resnet152 \
#                     -t "layer3" \
#                     -k 5 \
#                     -o "/content/gdrive/My Drive/UnB/TCC-1/TCC1-1-dataset-final/intrepretabilidade_results/TCC2" \
#                     -i "/content/gdrive/My Drive/UnB/TCC-1/TCC1-1-dataset-final/dataset_final/test/melanocytic-nevus/worst.jpg"

In [ ]:
#!python main.py demo2 -o "/content/gdrive/My Drive/UnB/TCC-1/TCC1-1-dataset-final/intrepretabilidade_results/TCC2" \
#                      -i "/content/gdrive/My Drive/UnB/TCC-1/TCC1-1-dataset-final/dataset_final/test/melanocytic-nevus/worst.jpg"

In [ ]:
#!python main.py demo4 -o "/content/gdrive/My Drive/UnB/TCC-1/TCC1-1-dataset-final/intrepretabilidade_results/TCC2" \
#                      -i "/content/gdrive/My Drive/UnB/TCC-1/TCC1-1-dataset-final/dataset_final/test/melanocytic-nevus/worst.jpg" \
#                      -k 3
#                      -a "resnet152"

# Função global para criar os resultados automatizados nos diretorios

In [23]:
import os 
import shutil

In [32]:
OUTPUT_DIR_BASE = "/content/gdrive/My Drive/UnB/TCC-1/TCC1-1-dataset-final/intrepretabilidade_results/complexidade/out"

In [25]:
# função para gerar os valores das camadas convolucionais
def results_layers_demo40(target_class, output_dir, input_imagem):
  output_dir = output_dir + "/demo40/" + str(target_class)  
  # create safely dir
  create_safe_dir(output_dir)

  print("Criando o demo40 para a imagem ", input_imagem)
  cmd = 'python main.py demo40 -c "{0}" \
                      -o "{1}" \
                      -i "{2}"'.format(target_class, output_dir, input_imagem)

  os.system(cmd)

In [26]:
# função para gerar os valores das camadas residuais com 3 tipos de grandscan
def results_layers_demo1(output_dir, input_imagem):
  output_dir = output_dir + "/demo1"

  # create safely dir
  create_safe_dir(output_dir)

  # Layers residuais
  layers = ["relu", "layer1", "layer2", "layer3", "layer4"]
  for layer in layers:
    print("Criando o demo1 para a imagem: {0} na camada: {1}".format(input_imagem, layer))
    cmd = 'python main.py demo1 -a resnet152 \
                      -t "{0}" \
                      -k 5 \
                      -o "{1}" \
                      -i "{2}"'.format(layer, output_dir, input_imagem)

    os.system(cmd)

In [27]:
# função para gerar os valores das camadas convolucionais
def results_layers_demo2(target_class, output_dir, input_imagem):
  output_dir = output_dir + "/demo2/" + str(target_class) 
  # create safely dir
  create_safe_dir(output_dir)

  print("Criando o demo2 para a imagem ", input_imagem)
  cmd = 'python main.py demo2 -c {0} \
                      -o "{1}" \
                      -i "{2}"'.format(target_class, output_dir, input_imagem)

  os.system(cmd)

In [28]:
# função de class probability
def results_class_probability(input_imagem, output_dir):
  # plot and save the class probability
  print("Criando a class probability da imagem", input_imagem)
  plot_bar(input_imagem, model, output_dir + "/class_probability")

In [29]:
# função para fazer a occlusion sensitivity map
def results_occlusion_sensitivity(output_dir, input_imagem):
  output_dir = output_dir + "/demo4"

  # create safely dir
  create_safe_dir(output_dir)

  cmd = 'python main.py demo4 -a "resnet152" \
                      -o "{0}" \
                      -i "{1}"'.format(output_dir, input_imagem)

  os.system(cmd)

In [30]:
#  função auxiliar para criar e remover diretorios
def create_safe_dir(directory):
  # se não existe cria
  if not os.path.exists(directory):
    os.makedirs(directory)
  else:
    # se existe remove e cria de novo
    shutil.rmtree(directory)
    os.makedirs(directory)

In [31]:
# traduzir o nome em numero das classes
def target_class_translation(file_full_path, INPUT_DIR_BASE):

  probs, classes = predict(file_full_path, model)

  true_class = os.path.basename(INPUT_DIR_BASE)

  dict_class = {'actinic-keratosis': 0,
  'basal-cell-carcinoma': 1,
  'dermatofibroma': 2,
  'hemangioma': 3,
  'intraepithelial-carcinoma': 4,
  'malignant-melanoma': 5,
  'melanocytic-nevus': 6,
  'pyogenic-granuloma': 7,
  'squamous-cell-carcinoma': 8}

  true_class = dict_class.get(true_class)
  prob_class = dict_class.get(classes[0])
  return true_class, prob_class

### chamada principal

In [34]:
INPUT_DIR_BASE = "/content/gdrive/My Drive/UnB/TCC-1/TCC1-1-dataset-final/dataset_final/test/pyogenic-granuloma"

In [46]:
%%time

for file in list(os.listdir(INPUT_DIR_BASE)):
  file_full_path = INPUT_DIR_BASE + "/" + file
  file_output_path_base = OUTPUT_DIR_BASE + "/" + file 

  true_class, prob_class = target_class_translation(file_full_path, INPUT_DIR_BASE)

  if str(file) != ".ipynb_checkpoints":
    # cria o dir de forma segura
    create_safe_dir(file_output_path_base)

    # 1 - results_class_probability
    #results_class_probability(file_full_path, file_output_path_base)

    # 2 - Demo 1
    #results_layers_demo1(file_output_path_base, file_full_path)

    # 4 - Demo 2 - true
    #results_layers_demo2(true_class, file_output_path_base, file_full_path)

    # 5 - Demo 2 - prob
    #results_layers_demo2(prob_class, file_output_path_base, file_full_path)

    # 6 - Demo 40 - true
    %memit results_layers_demo40(true_class, file_output_path_base, file_full_path)

    # 7 - Demo 40 - prob
    #results_layers_demo40(prob_class, file_output_path_base, file_full_path)

  else:
    print("Skip .ipynb_checkpoints")

  break


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  from ipykernel import kernelapp as app


Criando o demo40 para a imagem  /content/gdrive/My Drive/UnB/TCC-1/TCC1-1-dataset-final/dataset_final/test/pyogenic-granuloma/1338.jpg
peak memory: 3237.21 MiB, increment: 0.00 MiB
CPU times: user 112 ms, sys: 60.5 ms, total: 173 ms
Wall time: 7.88 s


In [ ]:
OUTPUT_DIR_BASE = "/content/gdrive/My Drive/UnB/TCC-1/TCC1-1-dataset-final/intrepretabilidade_results/TCC2/squamous-cell-carcinoma"
INPUT_DIR_BASE = "/content/gdrive/My Drive/UnB/TCC-1/TCC1-1-dataset-final/dataset_final/test/squamous-cell-carcinoma"

In [ ]:
for file in list(os.listdir(INPUT_DIR_BASE)):
  file_full_path = INPUT_DIR_BASE + "/" + file
  file_output_path_base = OUTPUT_DIR_BASE + "/" + file 

  true_class, prob_class = target_class_translation(file_full_path, INPUT_DIR_BASE)

  if str(file) != ".ipynb_checkpoints":
    # cria o dir de forma segura
    create_safe_dir(file_output_path_base)

    # 1 - results_class_probability
    results_class_probability(file_full_path, file_output_path_base)

    # 2 - Demo 1
    results_layers_demo1(file_output_path_base, file_full_path)

    # 4 - Demo 2 - true
    results_layers_demo2(true_class, file_output_path_base, file_full_path)

    # 5 - Demo 2 - prob
    results_layers_demo2(prob_class, file_output_path_base, file_full_path)

    # 6 - Demo 40 - true
    results_layers_demo40(true_class, file_output_path_base, file_full_path)

    # 7 - Demo 40 - prob
    results_layers_demo40(prob_class, file_output_path_base, file_full_path)

  else:
    print("Skip .ipynb_checkpoints")

Output hidden; open in https://colab.research.google.com to view.